## Asian Option Pricer

We have used the Python programming language to help us devise a model for pricing a European arithmetic Asian call option. If the Asian call option is positive, a pay between the asset price’s arithmetic average and the strike price at the expiry of the option. We collect the data from a series of asset prices which are then used to calculate the arithmetic average.

AT = arithmetic average of asset price
K = strike price
T = maturity date or expiry
Sti = asset price

$$ A_T = \frac{1}{N}\sum_{i=1}^{N}S_{t_i} $$

We will use the geometric average of the Asian option to price the Asian call option. While a geometric Asian call option does not exist in the market, there is a set formula for pricing it. Seeing that there is no formula to price the arithmetic Asian call option, the geometric Asian call option will be used as a way to pay the difference between the asset price’s geometric average and the strike price at expiry.
GT = geometric average of asset price

$$ G_T = \bigg(\prod_{i=1}^{N}S_{t_i}\bigg)^{\frac{1}{N}} $$

Import necessary libraries

In [2]:
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm
import scipy
import numpy as np

###### The standard Black-Scholes 
$$ C = SN(d_1)-N(d_2)Ke^{-rt}$$
 
$$ d_1 = \frac{ln(S/K)+(r+\frac{s^2}{2})t}{s\sqrt{t}}$$
 
$$ d_2 = d_1 - s\sqrt{t}$$

In [4]:
def BlackScholes(spot, strike, rate, vol, div, expiry):
    N = norm.cdf
    d1 = (np.log(spot/strike) + (rate - div - 0.5 * vol * vol) * expiry) / (vol * np.sqrt(expiry))
    d2 = d1 - vol * np.sqrt(expiry)
    call_price = spot * np.exp(-div * expiry) * N(d1) -  strike * np.exp(-rate * expiry) * N(d2)
    
    return call_price

The Geometric Asian Call uses the Standard Black-Scholes to price "true geometric price," which in our model is *Gstar.*

This works through a modification of the Black-Scholes function. 

$$C_{GA}=e^{-rt}\big(e^{(a+\frac{1}{2}b)}N(x)-KN(x-\sqrt{b})\big)$$
 
Where,

$$ a = ln(G_t) + \frac{N-m}{N}\bigg(ln(S)+v(t_{m+1}-t)+\frac{1}{2}v(T-t_{m+1})\bigg)$$
 
$$ b = -1 $$
 
$$ v = r-\delta-\frac{1}{2}\sigma^2$$
 
$$ x = \frac{a-ln(K)+b}{\sqrt{b}}$$

In [5]:
def GeometricAsianCall(spot, strike, rate, vol, div, expiry, num_steps):
    dt = expiry/num_steps
    nu = rate - div - 0.5 * vol * vol
    a = num_steps * (num_steps - 1) + (2.0 * num_steps + 1.0) / 6.0
    V = np.exp(-rate * expiry) * spot * np.exp(((num_steps + 1) * nu / 2.0 + vol * vol * a / (2.0 * num_steps * num_steps)) * dt)
    vang = vol * np.sqrt(a) / (pow(num_steps, 1.5))
    
    price = BlackScholes(V, strike, rate, vang, 0, expiry)
    
    return price

Call Payoff Function

In [6]:
def CallPayoff(spot, strike):
    return np.maximum(spot - strike, 0.0)

We enter the option parameters

* spot = Option Spot Price
* Strike = Option Strike Price
* rate = 3 Month Libor Rate
* vol = Volatility of Option
* expiry = Option Expiration Date
* num_reps = Number of Monte Carlo Simulations
* num_steps = Number of "Averaging Periods" before expiration
* b = beta, which is assumed to be -1

In [7]:
## Option parameters
spot = 40
strike = 40
rate = 0.08
vol = 0.3
div = 0.0
expiry = 0.24
num_reps = 1000
num_steps = 5
b = -1.0

Defining functions to be used in the Monte Carlo simulations

* z = provides the random walk that simulates market volatility
* drift and diffusion = are both used in the arithmetic Asian pricer used in the simulations
* disc = discounting variable to get the present value of the Asian Option
* Gstar = Calls the Geometric Asian Call function to get the "true geometric value of the option."
* spotT = Is a zero-filled matrix in created in preparation for Monte Carlo simulation
* spotT[:,i] = Fills the first column of every row with the original spot price
* A and G = Are zero-filled lists in preparation for Monte Carlo simulations

In [8]:
dt = expiry / num_steps
z = np.random.normal(size=(num_reps, num_steps))
drift = (rate - div - 0.5 * vol * vol) * dt
diffusion = vol * np.sqrt(dt) 
disc = np.exp(-rate * expiry)

Gstar = GeometricAsianCall(spot, strike, rate, vol, div, expiry, num_steps)
spotT = np.zeros((num_reps, num_steps))
spotT[:,0] = spot
A = np.zeros(num_reps)
G = np.zeros(num_reps)

Stepping into the Monte Carlo Simulation
* for i in range(num_reps) = is the function that runs each simulation 'num_reps' times
* for j in range(num_steps) = is the function that runs each step (j) per simulation (i)
* SpotT[i,j] = calculates the arithmetic Asian option price for every step in every simulation
* A_mean = is the arithmetic mean of the arithmetic prices determined in each (i) simulation
* G_mean = is the geometric mean of the arithmetic prices determined in each (i) simulation
* A = is the arithmetic mean of all the arithmetic means generated in the (num_reps) simulations
* G = is the arithmetic mean of all the geometric means generated in the (num_reps) simulations
* call_price = is the present value of the adjusted arithmetic mean
    * The "adjustment" is calculated by taking the difference between "true geometric price" Gstar and the mean of G calculated in the Monte Carlo simulations


In [9]:
for i in range(num_reps):
    for j in range(1, num_steps):
        spotT[i,j] = spotT[i,j-1] * np.exp(drift + diffusion * z[i,j])
        
    A_mean = spotT[i].mean()
    G_mean = pow(spotT[i].prod(), 1/num_steps)
    A[i] = CallPayoff(A_mean, strike)
    G[i] = CallPayoff(G_mean, strike)

In [10]:
call_price = disc * A.mean() + b * (Gstar - G.mean())

#stan_dev = np.std(callT)
#stan_err = scipy.stats.sem(callT)


print("Price: ${0:0.2f}".format(call_price))



#plt.plot(A)
#plt.axis([0,num_reps,0,20])
#plt.show()


Price: $1.47
